# LSTM with Word2Vec

In [22]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import string
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from assnat.params import *
from assnat.clean import complete_preproc
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Masking, Input
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

from gensim.models import Word2Vec
import fasttext

In [23]:
# Chargement et prétraitement des données
leg_ = 'data/leg16.csv'
min_words_=10

df = pd.read_csv(leg_)
df_preproc = complete_preproc(df, na_col=["Texte", "famille"], drop_names=["Mme la présidente", "M. le président"], min_words=min_words_, punct_opt=True)
df_preproc.to_csv('data/leg16_preproc.csv', index=False)

X = df_preproc['Texte']  # Les textes à classifier
y = df_preproc['famille']

# Encodage des labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
X.shape, len(y_encoded)

Columns dropped
family dropped
NaN dropped
Names dropped
Short sentences removed
Preprocessing done!


((131558,), 131558)

In [68]:
df_preproc.head()

,Unnamed: 0,Nom Orateur,ID Orateur,Texte,Thème Séance,ID Séance,Date Séance,ID Session,grammaire,famille
3,3,Mme Agnès Pannier-Runacher,759832,après la promulgation vendredi de la loi relat...,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,PAROLE_GENERIQUE,Centre-droit
5,5,Mme Agnès Pannier-Runacher,759832,comment se dire écologiste lorsqu’on ne vote p...,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,PAROLE_GENERIQUE,Centre-droit
7,7,Mme Agnès Pannier-Runacher,759832,…lorsqu’on vante le modèle allemand qui repose...,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,PAROLE_GENERIQUE,Centre-droit
8,8,Mme Julie Laernoes,794146,il n’y a pas eu de projet de loi sur cette que...,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,INTERRUPTION_1_10,Gauche
9,9,Mme Agnès Pannier-Runacher,759832,si nous voulons atteindre la neutralité carbon...,Accélération des procédures liées à la constru...,RUANR5L16S2023IDS26877,lundi 13 mars 2023,SCR5A2023O1,PAROLE_GENERIQUE,Centre-droit


In [24]:
max_words = 100000  # Limiter le nombre de mots à 100000
# oov_tok='<OOV>'

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# X_train = X_train.astype(str)
# X_test = X_test.astype(str)

# # Tokenization des textes
# tokenizer = Tokenizer(num_words=max_words, oov_token=oov_tok)
# tokenizer.fit_on_texts(X_train)
# X_train_seq = tokenizer.texts_to_sequences(X_train)
# X_test_seq = tokenizer.texts_to_sequences(X_test)

def split_text(text):
    return text.split()

X_train_split = X_train.apply(lambda x: split_text(x))
X_test_split = X_test.apply(lambda x: split_text(x))

# Embedding avec Word2Vec
word2vec = Word2Vec(sentences=X_train_split
                    , vector_size=100 # Dimension de l'espace vectoriel => IMPACT??? (ne change pas la dimension de l'embedding)
                    #, window=5, min_count=1, workers=4
                    )

In [25]:
X_train_split

252008    [au, cours, de, ce, quinquennat, nous, devrons...
296395    [s’ils, veulent, effectuer, correctement, leur...
293416    [motions, prises, de, parole, identiques, à, r...
233145    [il, est, urgent, monsieur, le, ministre, délé...
107178    [je, l’ai, évoqué, plus, tôt, nous, développon...
                                ...                        
130937    [en, effet, il, s’agit, d’une, société, étrang...
262656    [madame, la, première, ministre, à, la, veille...
284918    [voyez, ce, qui, se, passe, en, ce, moment, au...
247350    [c’est, pour, ça, que, nous, avons, dit, que, ...
289731    [elles, ne, sont, donc, pas, au, niveau, de, l...
Name: Texte, Length: 105246, dtype: object

In [26]:
#len(word2vec.wv.key_to_index), len(tokenizer.word_index) + 1

In [27]:
word2vec.wv.key_to_index

{'de': 0,
 'la': 1,
 'le': 2,
 'à': 3,
 'les': 4,
 'des': 5,
 'et': 6,
 'que': 7,
 'en': 8,
 'nous': 9,
 'vous': 10,
 'pas': 11,
 'du': 12,
 'qui': 13,
 'pour': 14,
 'un': 15,
 'je': 16,
 'dans': 17,
 'ce': 18,
 'ne': 19,
 'il': 20,
 'une': 21,
 'est': 22,
 'sur': 23,
 'par': 24,
 'au': 25,
 'a': 26,
 '!': 27,
 '–': 28,
 'plus': 29,
 'c’est': 30,
 'aux': 31,
 'sont': 32,
 'mais': 33,
 'cette': 34,
 'ou': 35,
 'ces': 36,
 'loi': 37,
 'notre': 38,
 'avec': 39,
 'se': 40,
 'tout': 41,
 'comme': 42,
 'été': 43,
 'votre': 44,
 '?': 45,
 'si': 46,
 'nos': 47,
 'bien': 48,
 'ont': 49,
 'leur': 50,
 'amendement': 51,
 'même': 52,
 'cet': 53,
 'fait': 54,
 'd’un': 55,
 'n’est': 56,
 'qu’il': 57,
 'ministre': 58,
 'y': 59,
 'donc': 60,
 'faire': 61,
 'd’une': 62,
 'avons': 63,
 'on': 64,
 '»': 65,
 '«': 66,
 'être': 67,
 'france': 68,
 'texte': 69,
 'cela': 70,
 'tous': 71,
 'très': 72,
 'monsieur': 73,
 'l’article': 74,
 'gouvernement': 75,
 'aussi': 76,
 'son': 77,
 'avez': 78,
 'question': 79

In [28]:
X_train_split[100]

['voilà',
 'ce',
 'qui',
 'vous',
 'importe',
 'couler',
 'le',
 'béton',
 'pour',
 'que',
 'l’installation',
 'puisse',
 'être',
 'inaugurée',
 'par',
 'le',
 'prince',
 'macron',
 '!']

In [29]:
def embed_sentence(word2vec, sentence):
    # $CHALLENGIFY_BEGIN
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
    return np.array(embedded_sentence)
    # $CHALLENGIFY_END

### Checks
embedded_sentence = embed_sentence(word2vec, X_train_split[100])
assert(type(embedded_sentence) == np.ndarray)
embedded_sentence.shape
# embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)
# assert(type(embedded_sentence_missing_words) == np.ndarray)
# assert(embedded_sentence_missing_words.shape == (3, 100))


(18, 100)

In [30]:
def embedding(word2vec, sentences):
    # $CHALLENGIFY_BEGIN
    embed = []
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
    return embed
    # $CHALLENGIFY_END
X_train_embedded = embedding(word2vec, X_train_split)
X_test_embedded = embedding(word2vec, X_test_split)

In [31]:
np.array(X_train_embedded).shape

/var/folders/mp/99nc3g2s3t192ft55tyrlzdh0000gn/T/ipykernel_5829/1703749569.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(X_train_embedded).shape


(105246,)

In [32]:
np.array(X_train_embedded)[100].shape

/var/folders/mp/99nc3g2s3t192ft55tyrlzdh0000gn/T/ipykernel_5829/153168504.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(X_train_embedded)[100].shape


(64, 100)

In [33]:
len(X_train_embedded)

105246

In [61]:
sample_size = len(X_train_embedded)
# $DELETE_BEGIN
X_train_pad = pad_sequences(X_train_emb_fr[0:sample_size+1], dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_emb_fr[0:sample_size+1], dtype='float32', padding='post')

# $DELETE_EN'
len(X_train_pad), X_train_pad[100].shape

(105246, (300,))

In [15]:
len(np.unique(y_encoded))

8

In [62]:
# Construire le modèle RNN avec LSTM
#embedding_dim = embedding_dim_
patience_ = 20
epoch_=5

early_stopper = EarlyStopping(monitor='val_loss', patience=patience_, restore_best_weights=True)

model = Sequential()
#model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
#model.add(SpatialDropout1D(0.2))

model.add(Input((1,300)))
model.add(Masking())
model.add(LSTM(256, activation = 'tanh', dropout=0.2))
#model.add(Dense(64, activation='relu'))
model.add(Dense(len(np.unique(y_encoded)), activation='softmax'))

# Compiler le modèle
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# Afficher un résumé du modèle
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_6 (Masking)         (None, 1, 300)            0         
                                                                 
 lstm_6 (LSTM)               (None, 256)               570368    
                                                                 
 dense_8 (Dense)             (None, 8)                 2056      
                                                                 
Total params: 572,424
Trainable params: 572,424
Non-trainable params: 0
_________________________________________________________________
None


In [64]:
X_train_pad =np.expand_dims(X_train_pad, 1)

In [65]:
X_train_pad.shape

(105246, 1, 300)

In [66]:
X_test_pad = np.expand_dims(X_test_pad, 1)

In [67]:
# Entraîner le modèle
batch_size = 64
epochs = 100

history = model.fit(
    X_train_pad, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test_pad, y_test),
    callbacks=[early_stopper],
    verbose=1
)

# Prédire sur l'ensemble de test
y_pred = model.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred, axis=1)

# Afficher le rapport de classification
print("Rapport de classification pour le RNN avec LSTM:")
print(classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_))

#train_model_RNN(leg_='data/leg16.csv', min_words_=10, patience_=3, epoch_=1, embedding_dim_=100, max_len_=100)

Epoch 1/100
1645/1645 [==============================] - 25s 12ms/step - loss: 1.7993 - accuracy: 0.3261 - val_loss: 1.7810 - val_accuracy: 0.3251
Epoch 2/100
1645/1645 [==============================] - 19s 12ms/step - loss: 1.7778 - accuracy: 0.3276 - val_loss: 1.7673 - val_accuracy: 0.3278
Epoch 3/100
1645/1645 [==============================] - 17s 10ms/step - loss: 1.7707 - accuracy: 0.3288 - val_loss: 1.7611 - val_accuracy: 0.3282
Epoch 4/100
1645/1645 [==============================] - 17s 11ms/step - loss: 1.7648 - accuracy: 0.3293 - val_loss: 1.7572 - val_accuracy: 0.3273
Epoch 5/100
1645/1645 [==============================] - 17s 11ms/step - loss: 1.7620 - accuracy: 0.3289 - val_loss: 1.7552 - val_accuracy: 0.3276
Epoch 6/100
1645/1645 [==============================] - 17s 11ms/step - loss: 1.7606 - accuracy: 0.3296 - val_loss: 1.7532 - val_accuracy: 0.3286
Epoch 7/100
1645/1645 [==============================] - 17s 11ms/step - loss: 1.7592 - accuracy: 0.3305 - val_loss: 1

/Users/aaronviviani/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aaronviviani/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/aaronviviani/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control th

In [ ]:
from huggingface_hub import hf_hub_download
model_path = hf_hub_download(repo_id='facebook/fasttext-fr-vectors', filename='model.bin')
model = fasttext.load_model(model_path)
model.words

[',',
 'de',
 '.',
 '</s>',
 'la',
 'et',
 ':',
 'à',
 'le',
 '"',
 'en',
 '’',
 'les',
 'des',
 ')',
 '(',
 'du',
 'est',
 'un',
 "l'",
 "d'",
 'une',
 'pour',
 '/',
 '|',
 'dans',
 'sur',
 'que',
 'par',
 'au',
 'a',
 'l',
 'qui',
 '-',
 'd',
 'il',
 'pas',
 '!',
 'avec',
 '_',
 'plus',
 "'",
 'Le',
 'ce',
 'ou',
 'La',
 'ne',
 'se',
 '»',
 '...',
 '?',
 'vous',
 'sont',
 'son',
 '«',
 'je',
 'Les',
 'Il',
 'aux',
 '1',
 ';',
 'mais',
 "qu'",
 'on',
 "n'",
 'comme',
 '2',
 'sa',
 'cette',
 'y',
 'nous',
 'été',
 'tout',
 'fait',
 'En',
 "s'",
 'bien',
 'ses',
 'très',
 'ont',
 's',
 'être',
 'votre',
 'ai',
 'elle',
 'n',
 '3',
 'même',
 "L'",
 'deux',
 'faire',
 "c'",
 'aussi',
 '>',
 'leur',
 '%',
 'si',
 'entre',
 'qu',
 '€',
 '&',
 '4',
 'sans',
 'Je',
 "j'",
 'était',
 '10',
 'autres',
 'tous',
 'peut',
 'France',
 'ces',
 '…',
 '5',
 'lui',
 'me',
 ']',
 '[',
 'où',
 'ans',
 '6',
 '#',
 'après',
 '+',
 'ils',
 'dont',
 'Pour',
 '°',
 '–',
 'temps',
 '*',
 'sous',
 'Un',
 'avoir

In [ ]:
len(model.words)

2000000

In [ ]:
model[X_train[100]]

array([ 1.09213521e-03,  3.75561556e-03, -4.50983504e-03, -9.30140167e-03,
       -5.94677171e-03,  2.75583565e-03,  1.67043936e-02,  9.74733499e-04,
       -1.68547779e-03, -1.87480710e-02,  4.21768799e-03, -1.05491029e-02,
       -3.79482261e-03,  2.98430939e-04, -4.52585006e-03,  6.68866793e-03,
        8.24901741e-03, -2.66030547e-04, -5.75346779e-03,  1.00829490e-02,
       -4.17685456e-04, -7.06833275e-03, -1.45621144e-03, -4.61566122e-03,
        8.00048001e-03, -7.48887984e-03, -4.97669401e-03, -6.37914520e-04,
        2.17586644e-02,  1.93930417e-03,  9.40109428e-04,  1.06456866e-02,
        9.00505576e-03, -1.28913682e-03, -1.43216830e-03, -1.60469848e-03,
        6.48008566e-03,  4.76042414e-03, -2.66306242e-03,  1.15520842e-02,
       -7.21494248e-03, -4.21453081e-03, -6.47720008e-04, -4.03778162e-03,
       -7.70773832e-03,  3.15086581e-02, -4.90734565e-05,  1.06794760e-02,
        5.78911696e-03,  5.22339251e-03,  9.18255094e-03, -1.25983031e-02,
       -8.45877547e-03,  

In [39]:
X_train_emb_fr = np.array([model[x] for x in X_train])

In [40]:
X_test_emb_fr = np.array([model[x] for x in X_test])

In [ ]:
X_train_emb_fr.shape

(105246, 300)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from huggingface_hub import hf_hub_download
import fasttext

# Load FastText embeddings
model_path = hf_hub_download(repo_id='facebook/fasttext-fr-vectors', filename='model.bin')
fasttext_model = fasttext.load_model(model_path)

# Convert text data to FastText embeddings
def convert_to_embeddings(text_data):
    return np.array([np.array([fasttext_model.get_word_vector(word) for word in sentence]) for sentence in text_data])

# Convert the training and test sets to embeddings
X_train_emb_fr = convert_to_embeddings(X_train)  # Assuming X_train is a list of tokenized sentences
X_test_emb_fr = convert_to_embeddings(X_test)

# Pad the sequences
max_len = 100  # Define the maximum length based on your dataset
X_train_pad = pad_sequences(X_train_emb_fr, maxlen=max_len, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_emb_fr, maxlen=max_len, dtype='float32', padding='post')

# Define the LSTM model
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(max_len, 300)))  # 300 is the embedding dimension
model.add(LSTM(100, return_sequences=False))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Assuming label_encoder has been defined

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Set training parameters
batch_size = 64
epochs = 5  # You can change this to `epoch_` if it's a parameter

# Define Early Stopping
early_stopper = EarlyStopping(monitor='val_loss', patience=3)

# Train the model
history = model.fit(
    X_train_pad, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test_pad, y_test),
    callbacks=[early_stopper],
    verbose=1
)

# Predict on the test set
y_pred = model.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred, axis=1)

# Generate classification report
print("Classification Report for RNN with LSTM:")
print(classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_))


2024-08-26 17:10:22.192112: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


NameError: name 'X_train' is not defined